# Meeting Preperation with Slack Notification and Trello Board Creation using Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/meeting_prep_using_slack_and_trello_agents.ipynb)

## Dependencies

In [85]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null
%pip install duckduckgo-search > /dev/null
%pip install python-dotenv > /dev/null
%pip install slack_sdk > /dev/null
%pip install python-dotenv > /dev/null

## Tools

In [86]:
#ToDO: Code has been tested without Slack or Trello SDK, Carry out the testing once the integration of Slack and Trello is completed

import os
import requests
from bs4 import BeautifulSoup
from praisonai_tools import BaseTool
from langchain.tools import tool
from duckduckgo_search import DDGS
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
from typing import List

load_dotenv()


class InternetSearchTool(BaseTool):
    name: str = "InternetSearchTool"
    description: str = "Searches the internet for relevant information based on a query or for the latest news."

    def _run(self, query: str):
        ddgs = DDGS()
        results = ddgs.text(keywords=query, region='wt-wt', safesearch='moderate', max_results=5)
        return results


class SlackNotificationTool(BaseTool):
    name: str = "SlackNotificationTool"
    description: str = "Sends notifications to a Slack channel."

    def _run(self, text: str):
        # Get Slack token and channel ID from environment variables
        SLACK_TOKEN = os.getenv("SLACK_TOKEN")
        SLACK_CHANNEL_ID = os.getenv("SLACK_CHANNEL_ID")
        client = WebClient(token=SLACK_TOKEN)

        try:
            # Send a message to the Slack channel
            response = client.chat_postMessage(channel=SLACK_CHANNEL_ID, text=text)
            print("Message sent successfully!")
            return response
        except SlackApiError as e:
            error_message = f"Error sending message: {e.response['error']}"
            print(error_message)
            return error_message


class TrelloIntegrationTool(BaseTool):
    name: str = "TrelloIntegrationTool"
    description: str = "Creates Trello cards based on tasks provided, adding them to a specified Trello list."

    def _run(self, tasks: List[dict]):
        """
        Accepts a list of tasks and creates a Trello card for each task.
        Each task should be a dictionary with 'name' and 'description' keys.
        """
        API_KEY = os.getenv("TRELLO_API_KEY")
        TOKEN = os.getenv("TRELLO_TOKEN")
        LIST_ID = os.getenv("TRELLO_LIST_ID")

        def create_trello_card(task_title, task_description):
            url = "https://api.trello.com/1/cards"
            query = {
                "key": API_KEY,
                "token": TOKEN,
                "idList": LIST_ID,
                "name": task_title,
                "desc": task_description,
            }
            response = requests.post(url, params=query)
            if response.status_code == 200:
                print(f"Task '{task_title}' successfully created in Trello.")
                return response
            else:
                error_message = f"Failed to create task '{task_title}' in Trello: {response.text}"
                print(error_message)
                return error_message

        for task in tasks:
            task_title = task.get("name")
            task_description = task.get("description")
            if task_title and task_description:
                create_trello_card(task_title, task_description)
            else:
                print("Task missing title or description. Skipping...")


## YAML Prompt

In [87]:
agent_yaml = """
framework: "crewai"
topic: "Meeting Preparation and Task Analysis"
roles:
  meeting_analyzer:
    role: "Meeting Transcript Analysis Agent"
    backstory: |
      An expert in analyzing meeting transcripts to identify and extract actionable tasks and issues.
      Known for breaking down complex discussions into structured tasks that teams can follow up on effectively.
    goal: "Analyze the meeting transcript to extract tasks and actionable items."
    tasks:
      analyze_meeting:
        description: "Analyze the provided meeting transcript to identify tasks, issues, and relevant action items. Organize the findings into actionable points with clear details and criteria for each task."
        expected_output: "JSON-formatted list of tasks, including titles, descriptions, and detailed criteria for each task."
    tools:
      - "InternetSearchTool"
      - "SlackNotificationTool"
      - "TrelloIntegrationTool"
dependencies: []
"""

## Main

In [89]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[InternetSearchTool])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 5/10


 [2024-11-04 07:48:34][DEBUG]: == Working Agent: Meeting Transcript Analysis Agent
 [2024-11-04 07:48:34][INFO]: == Starting Task: Analyze the provided meeting transcript to identify tasks, issues, and relevant action items. Organize the findings into actionable points with clear details and criteria for each task.


> Entering new CrewAgentExecutor chain...
I need to analyze the provided meeting transcript to extract actionable tasks and issues. However, I currently do not have access to the transcript itself. I should first search for a relevant meeting transcript or any necessary information that could help me understand the context or topics discussed in the meeting.

Action: InternetSearchTool
Action Input: {"query": "recent meeting transcripts on project updates or team discussions"}


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': '4 Examples: Meeting Recap Email Template (with Tips)', 'href': 'https://status.net/articles/meeting-recap-email-template-4-examples-tips/', 'body': "Meeting Recap Email Example for a Team Meeting. Subject: Recap of Our February 9 Team Meeting. Hi Team, Thanks for your great input and discussion during today's meeting. Here's a quick recap of the key points and actions to take: Project X Update: We're ahead of schedule, and Sarah will circulate the updated timeline by Feb 11."}, {'title': 'Meeting Recap 101: Templates, Examples & Tools | Dive', 'href': 'https://www.letsdive.io/blog/how-to-write-a-meeting-recap', 'body': 'There are several meeting productivity tools available that can help you write effective meeting recaps. Here are a few examples: 1. Project Management Tools: Tools like Trello, Asana, and Basecamp can help you keep track of action items discussed during meetings and assign them to team members with due dates. 2.'}, {'title': 'How To Send a Meeting Recap 

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': '4 Examples: Meeting Recap Email Template (with Tips)', 'href': 'https://status.net/articles/meeting-recap-email-template-4-examples-tips/', 'body': '1. Subject Line. Your subject line should be clear and indicative of the email content: "Meeting Recap - [Meeting Topic] - [Date]". This lets recipients immediately know the purpose of the email. 2. Opening Greeting. Start with a simple greeting, like "Hi team," or "Dear [Team Name],". This sets a warm tone for the email and ...'}, {'title': 'Project Status Update Email Examples and Samples Included - Woculus', 'href': 'https://www.woculus.com/email-sharing-project-status-update/', 'body': "Progress Update Email Sample. Subject: Project Alpha Progress Update. Dear Team, I wanted to provide a brief progress update on Project Alpha. We have made significant strides since our last update and I am pleased with the team's dedication and hard work. Key Developments: 1."}, {'title': 'How to Nail Your Next Project Status Update - Me

...

 

[{'title': 'PDF PROJECT MANAGEMENT MEETING MINUTES TEMPLATE SAMPLE - Smartsheet', 'href': 'https://www.smartsheet.com/sites/default/files/2023-10/IC-Project-Management-Meeting-Minutes-Sample-11856_PDF.pdf', 'body': 'PROJECT MANAGEMENT MEETING MINUTES TEMPLATE SAMPLE. 1. MEETING DETAILS. DATE LOCATION START TIME END TIME ATTENDEES NAME . ... Sign below to approve meeting minutes. PARTICIPANT NAME . SIGNATURE . DISCLAIMER . Any articles, templates, or information provided by Smartsheet on the website are for ... 9/13/2023 10:32:32 PM ...'}, {'title': 'Automate note-taking in Microsoft Teams with meeting transcription', 'href': 'https://techcommunity.microsoft.com/t5/microsoft-mechanics-blog/automate-note-taking-in-microsoft-teams-with-meeting/ba-p/2242782', 'body': 'QUICK LINKS: 00:23 — Meeting notes capability in Microsoft Teams. 01:00 — Automated meeting transcription demo. 02:13 — Finding the meeting transcript after the meeting. 02:55 — Comparison with automatic captions in Micros

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'Project Management Meetings: The Definitive 2023 Guide - adam.ai', 'href': 'https://adam.ai/blog/project-management-meetings-guide', 'body': 'adam.ai is trusted and used by powerful teams and organizations worldwide for all types of critical meetings, like board, committee, project management, and business development meetings. And most importantly, adam.ai integrates with your existing workflow, is SOC2 compliant, provides dedicated support and success, and has a free trial option.'}, {'title': 'Free Project Management Meeting Agenda Templates', 'href': 'https://www.smartsheet.com/content/project-management-agenda-templates', 'body': "Download a Project Governance Meeting Agenda Template for. Microsoft Word | Google Docs. This project governance meeting agenda template helps ensure clear oversight, strategic alignment, and effective decision-making for a project's direction and key milestones. The template serves as a guide for overseeing big project decisions and lists

### Task Output ###
```json
[
    {
        "title": "Update Project Timeline",
        "description": "Review and update the project timeline to reflect current progress and any delays.",
        "criteria": {
            "responsible": "Project Manager",
            "due_date": "2023-10-15",
            "details": "Ensure all team members review the updated timeline and provide feedback."
        }
    },
    {
        "title": "Conduct Team Check-In",
        "description": "Schedule a team check-in to discuss individual progress and address any blockers.",
        "criteria": {
            "responsible": "Team Lead",
            "due_date": "2023-10-10",
            "details": "Gather input from all team members on current tasks and obstacles."
        }
    },
    {
        "title": "Budget Review Meeting",
        "description": "Hold a budget review meeting to assess current expenditures and forecast future spending.",
        "criteria": {
            "responsible": "Finance Officer",
            "due_date": "2023-10-20",
            "details": "Prepare a report on current budget status and anticipated financial needs for the next 
quarter."
        }
    },
    {
        "title": "Develop Marketing Strategy",
        "description": "Create a comprehensive marketing strategy for the next project phase.",
        "criteria": {
            "responsible": "Marketing Manager",
            "due_date": "2023-10-25",
            "details": "Include target audience analysis, key messaging, and promotional tactics."
        }
    },
    {
        "title": "Risk Assessment Update",
        "description": "Update the risk assessment document to include new risks identified in recent project 
phases.",
        "criteria": {
            "responsible": "Risk Manager",
            "due_date": "2023-10-30",
            "details": "Ensure all identified risks are documented and mitigation strategies are suggested."
        }
    },
    {
        "title": "Client Feedback Session",
        "description": "Schedule a session to gather feedback from clients on the latest project deliverables.",
        "criteria": {
            "responsible": "Client Relations Manager",
            "due_date": "2023-11-05",
            "details": "Prepare a set of questions to guide the discussion and capture client insights."
        }
    }
]
```

None
